In [2]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

### Uploading & Cleaning Tweets

In [5]:
tweets = pd.read_csv('tweets.csv', usecols = ['timestamp', 'text'],encoding='latin1')  #nrows 'in_reply_to_status_id'

In [6]:
#Add cols for month and year

index = 0;

tweets['month'] = 0
tweets['year'] = 0

for i in tweets['timestamp']:
    tweets['year'][index]=tweets['timestamp'][index][0:4]
    tweets['month'][index]=tweets['timestamp'][index][5:7]
    index+=1

#Delete timestamp column
del tweets['timestamp']

C:\Users\aliae\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\aliae\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [7]:
#Convert to lowercase
tweets['text'] = tweets['text'].str.lower()

In [8]:
#Clean up - remove replies, retweets

tweets = tweets[~tweets.text.str.contains("@")]
tweets = tweets[~tweets.text.str.contains("rt")]
#tweets = tweets[~tweets.text.str.contains("plx11zpn21")]

#Remove all non-letter or number chars
tweets['text'] = tweets['text'].replace('[^a-zA-Z0-9]',' ', regex = True) #replace non letters or numbers

#Split data by year (2011 - 2014) & sort months in descending order
def tweets_by_year(year):
    is_year = tweets['year'] == year
    return tweets[is_year].sort_values(by=['month'])

tweets_2011 = tweets_by_year(2011)
tweets_2012 = tweets_by_year(2012)
tweets_2013 = tweets_by_year(2013)
tweets_2014 = tweets_by_year(2014)

In [11]:
'''wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'white',
    stopwords = STOPWORDS
).generate(str(tweets_x)
)
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()'''

### Convert data from daily to monthly

In [12]:
def df_to_text(tweets):
    return tweets.text.str.cat(sep=' ')


from collections import Counter

#split_it = tweets_2011_t.split() 
#Counter = Counter(split_it) 
#print(Counter.most_common(100))

In [14]:
#Reset index

tweets_2011 = tweets_2011.reset_index()
del tweets_2011['index']

tweets_2012 = tweets_2012.reset_index()
del tweets_2012['index']

tweets_2013 = tweets_2013.reset_index()
del tweets_2013['index']

tweets_2014 = tweets_2014.reset_index()
del tweets_2014['index']

In [16]:
from collections import Counter


#Create new dataset that has all the text for tweets within 1 month in the same row, 
#while maintaining other metadata

#Find most common words for each month

def tweets_by_month(tweets_year):
    stop_words = STOPWORDS
    num_rows = len(tweets_year.index)
    by_month = pd.DataFrame(index=np.arange(12), columns=['year', 'month', 'count','text','common_words'])
    
    by_month['month'][0]=tweets_year['month'][0]
    by_month['year'][0] = tweets_year['year'][0] 
    by_month['text'][0] = tweets_year['text'][0]
    by_month['count'][0] = 1
    
    ind_m = 0 
    ind_y = 1
    
    for i in range(num_rows-1):
        if tweets_year['month'][ind_y] > by_month['month'][ind_m]: #initialize new month
            
            #Find common words that aren't stopwords
            split_it = by_month['text'][ind_m].split() 
            count = Counter(split_it) 
            for i in split_it:
                words = [w for w in split_it if w not in STOPWORDS and len(w) > 3]
                count.update(words)
            by_month['common_words'][ind_m]=count
            
            ind_m+=1
            by_month['month'][ind_m] = tweets_year['month'][ind_y]
            by_month['year'][ind_m] = tweets_year['year'][ind_y] 
            by_month['text'][ind_m] = tweets_year['text'][ind_y]
            by_month['count'][ind_m]=1
        else:
            
            by_month['count'][ind_m]+=1
            by_month['text'][ind_m]= by_month['text'][ind_m] + '; ' + tweets_year['text'][ind_y]
        ind_y = ind_y +1
    
    return by_month
    
by_month_2011 = tweets_by_month(tweets_2011)
by_month_2012 = tweets_by_month(tweets_2012)
by_month_2013 = tweets_by_month(tweets_2013)
by_month_2014 = tweets_by_month(tweets_2014)

#Concat into 1 dataset
tweets_by_month = pd.concat([by_month_2011, by_month_2012, by_month_2013, by_month_2014])

#Save to CSV
tweets_by_month.to_csv(r'tweets_by_month.csv')

#Made some changes directly in excel, saved the new file as tweets_by_volume

In Excel, I used the common words output and monthly tweet text to interpret each month's emotions on a scale of 1-8. I also added a volume column, mapping to 1-4 based on the amount of tweets written per month (calculated as 'count' in the function above)

In [25]:
tweets_2011.head()

,text,month,year
0,masr lt 3,2,2011
1,school spirit lt 3,3,2011
2,grade seven school spirit,3,2011
3,walk for nile was such a succesful event,5,2011
4,...,5,2011


### Mapping Tweets to Pitches

In [34]:
#tweets_by_volume is the modified version of the previous tweets_by_month

volume = pd.read_csv('tweets_by_volume.csv', encoding='latin1')  #nrows 'in_reply_to_status_id'


In [35]:
volume.head()

,year,month,count,volume,pitch
0,2011,2,1,1,6
1,2011,3,2,1,5
2,2011,4,0,0,0
3,2011,5,8,1,5
4,2011,6,53,3,6


In [36]:
#Function to convert a csv file column to a single strong

def csv_to_text(volume3,label): 
    pitches = '1,'
    
    ind = 0
    
    for i in volume3[label]:
        pitches = str(pitches) + str(' ') + str(volume3[label][ind])
        ind+=1
    pitches = pitches + ';'
    return pitches

In [37]:
index = 0

#List of pitches I want to map to, 1 - 8

ch1 = [30,35]
ch2 = [37,40]
ch3 = [42,45]
ch4 = [47,48,50]
ch5 = [52,57,59]
ch6 = [60,62,69]
ch7 = [69,70,72]
ch8 = [75,78]

#to add a month text column
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

#each array entry maps to 1 - 8 and includes the different possible pitch outputs for said emotion category
choices = [ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8]

#new dataframe to store the pitches
newdata = pd.DataFrame(index=np.arange(236), columns=['index', 'pitch', 'channels', 'year', 'month', 'months_text'])

import random
vol_index = 0
index = 0

for i in volume['count']:
    #5 pitch outputs for each month to populate more data
    for i in range(5):
        newdata['index'][index] = vol_index+1
        #correlate 1-8 ranking to a random sequence of 5 pitches from the specific array index
        newdata['pitch'][index] = random.choice(choices[(volume['pitch'][vol_index])-1])
        newdata['channels'][index] = volume['volume'][vol_index]
        newdata['year'][index] = volume['year'][vol_index]
        newdata['month'][index] = volume['month'][vol_index]
        #add month text column
        newdata['months_text'][index] = months[(newdata['month'][index])-1]
        index+=1
    vol_index+=1

    
    
newdata.head(10)

,index,pitch,channels,year,month,months_text
0,1,60,1,2011,2,february
1,1,62,1,2011,2,february
2,1,62,1,2011,2,february
3,1,69,1,2011,2,february
4,1,62,1,2011,2,february
5,2,59,1,2011,3,march
6,2,59,1,2011,3,march
7,2,59,1,2011,3,march
8,2,52,1,2011,3,march
9,2,59,1,2011,3,march


### Export files for use in Max MSP patch

In [48]:
new_pitches = open("max_input/new_pitches.txt", "w")
n = new_pitches.write(csv_to_text(newdata,'pitch'))
new_pitches.close()

In [49]:
new_year = open("max_input/new_year.txt", "w")
n = new_year.write(csv_to_text(newdata,'year'))
new_year.close()

new_month = open("max_input/new_month.txt", "w")
n = new_month.write(csv_to_text(newdata,'month'))
new_month.close()

new_month_text = open("max_input/new_month_text.txt", "w")
n = new_month_text.write(csv_to_text(newdata,'months_text'))
new_month_text.close()

new_channels = open("max_input/new_channels.txt", "w")
n = new_channels.write(csv_to_text(newdata,'channels'))
new_channels.close()


In [40]:
print(csv_to_text(newdata,'month'))

1, 2 2 2 2 2 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5 6 6 6 6 6 7 7 7 7 7 8 8 8 8 8 9 9 9 9 9 10 10 10 10 10 11 11 11 11 11 12 12 12 12 12 1 1 1 1 1 2 2 2 2 2 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5 6 6 6 6 6 7 7 7 7 7 8 8 8 8 8 9 9 9 9 9 10 10 10 10 10 11 11 11 11 11 12 12 12 12 12 1 1 1 1 1 2 2 2 2 2 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5 6 6 6 6 6 7 7 7 7 7 8 8 8 8 8 9 9 9 9 9 10 10 10 10 10 11 11 11 11 11 12 12 12 12 12 1 1 1 1 1 2 2 2 2 2 3 3 3 3 3 4 4 4 4 4 5 5 5 5 5 6 6 6 6 6 7 7 7 7 7 8 8 8 8 8 9 9 9 9 9 10 10 10 10 10 11 11 11 11 11 12 12 12 12 12 nan;


In [41]:
#Save to CSV file
newdata.to_csv(r'newdata.csv')

In [42]:
#Made some changes in excel, added a track column that has either 1, 2, 3, or 4 depending on which additional track will be playing
#1 is protests, 2 is coldplay, 3 is doctor who, 4 is wambach scoring a goal - they correlate to the most tweeted about topics
newdata2 = pd.read_csv('newdata2.csv', encoding='latin1')  #nrows 'in_reply_to_status_id'
print(newdata2)

     Unnamed: 0  index  pitch  channels  year  month months_text  track
0             0      1     60         1  2011      2    february      1
1             1      1     60         1  2011      2    february      0
2             2      1     62         1  2011      2    february      0
3             3      1     69         1  2011      2    february      0
4             4      1     60         1  2011      2    february      0
..          ...    ...    ...       ...   ...    ...         ...    ...
230         230     47     37         1  2014     12    december      0
231         231     47     40         1  2014     12    december      0
232         232     47     40         1  2014     12    december      0
233         233     47     40         1  2014     12    december      0
234         234     47     40         1  2014     12    december      0

[235 rows x 8 columns]


In [47]:
tracks = open("max_input/new_tracks.txt", "w")
n = tracks.write(csv_to_text(newdata2,'track'))
tracks.close()

## Feed txt files into Max Patch

* Pitch --> determine pitch of current note
* Channels --> number of audio channels playing at the same time
* Tracks --> determine which additional track to play at current index
* Month & Year --> Output as text for visual reference of where in my tweet history the sonification is
